<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/crew_ai/resume_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Install required pacakges


In [3]:
!pip install --upgrade --quiet  langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.7 MB/s eta 0:00:00


In [4]:
!pip install -q -U crewai
!pip install -q -U duckduckgo-search
!pip install -q -U crewai_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 327.1/327.1 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 87.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.8/138.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.2 MB/s eta 

### Required Packages

In [5]:
import os

from crewai import Agent, Task, Crew, Process, LLM
#from crewai_tools import tool
from crewai.tools import BaseTool
from crewai_tools import CSVSearchTool, FileReadTool

/usr/local/lib/python3.11/dist-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'required'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool
from crewai.knowledge.source.pdf_knowledge_source import PDFKnowledgeSource

In [7]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
os.environ["GEMINI_API_KEY"] = GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
if GOOGLE_API_KEY :
  print("GOOGLE_API_KEY is set")

GOOGLE_API_KEY is set


### set up the LLM

In [39]:
#from crewai import Agent, LLM

gemini = LLM(
    api_key=os.getenv("GEMINI_API_KEY"),
    model="gemini/gemini-2.5-flash",
)

In [40]:
my_llm=gemini

### setup tools

In [41]:
cv_reader = Agent(
    role = 'CV Reader',
    goal = '''Extract relevant information from the CV, such as skills, experience, and education.''',
    backstory = '''With years of experience in HR, you excel at quickly identifying key qualifications in resumes.''',
    llm = my_llm,
    tools=[FileReadTool()],
    #knowledge_sources=[resume_pdf],
    verbose=True,
)

In [42]:
job_opportunities_parser = Agent(
    role = 'Job Opportunities Parser',
    goal = '''Extract job descriptions from the CSV file, including job title, required skills, and responsibilities.''',
    backstory = '''A data analyst who has transitioned into HR, you have a knack for organizing and interpreting job data.''',
    llm = my_llm,
    #knowledge_sources=[resume_pdf],
    tools=[ScrapeWebsiteTool()],
    verbose=True,
)

In [43]:
Matcher = Agent(
    role = 'Matcher',
    goal = '''Research companies and prepare interview insights''',
    backstory = '''You are a corporate research expert who excels at gathering and analyzing
    the latest company information. You know how to find and synthesize data
    from various sources to create comprehensive company profiles and prepare
    candidates for interviews.''',
    tools=[FileReadTool(), CSVSearchTool()],
    #knowledge_sources=[resume_pdf],
    llm = my_llm,
    verbose=True,
)

/usr/local/lib/python3.11/dist-packages/alembic/config.py:577: DeprecationWarning: No path_separator found in configuration; falling back to legacy splitting on spaces, commas, and colons for prepend_sys_path.  Consider adding path_separator=os to Alembic config.
  util.warn_deprecated(


### Tasks to perform

In [44]:
# Create tasks for your agents
read_cv_task = Task(
  description="""Extract relevant information from the given CV. Focus on skills, experience,
    education, and key achievements.
    Ensure to capture the candidate's professional summary, technical skills,
    work history, and educational background.


    CV file: {path_to_cv}
""",
  agent=cv_reader,
  verbose=True,
  #output_file='output/job_analysis.json',
  expected_output="""A structured summary of the CV, including:
    - Professional Summary
    - Technical Skills
    - Work History
    - Education
    - Key Achievements"""

)

In [57]:
match_cv_task = Task(
  description="""Match the CV to the job opportunities based on skills, experience, and key
    achievements.
    Evaluate how well the candidate's profile fits each job description,
    focusing on the alignment of skills, work history, and key achievements
    with the job requirements.


    Jobs CSV file: {path_to_jobs_csv}

    CV file: {path_to_cv}
    write the output in markdown format
    """,
  agent=Matcher,
  output_file='output/resume_match.md',
  expected_output="""A ranked list of job opportunities that best match the CV, including:
    - Job Title
    - Match Score (based on skills and experience. Max score is 10)
    - Key Matching Points""",
  context=[read_cv_task],
  #output_file= "output/{module_name}"
)

### Create a Crew

In [58]:
# Instantiate your crew with a sequential process
crew = Crew(
  agents=[cv_reader,job_opportunities_parser,Matcher],
  tasks=[read_cv_task,match_cv_task],
  #knowledge_sources=[resume_pdf],
  verbose=False, # You can set it to 1 or 2 to different logging levels
)

In [59]:
crew

Crew(id=8ae085e3-f46a-45f5-a153-e0f5f6021b83, process=Process.sequential, number_of_agents=3, number_of_tasks=2)

### Kickoff the crew - let the magic happen

In [60]:
inputs = {
        'path_to_jobs_csv': '/content/drive/MyDrive/github_projects/data_dir/resumes/data/jobs.csv',
        'path_to_cv': '/content/drive/MyDrive/github_projects/data_dir/resumes/data/cv.md'
    }

In [61]:
# Get your crew to work!
#result = crew.kickoff()
result = crew.kickoff(inputs=inputs)

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: CV Reader                                                                                               │
│                                                                                                                 │
│  Task: Extract relevant information from the given CV. Focus on skills, experience,                             │
│      education, and key achievements.                                                                           │
│      Ensure to capture the candidate's professional summary, technical skills,                                  │
│      work history, and educational background.                                                                  │
│                                                                                                                 │
│                                                                                                                 │
│      CV file: /content/drive/MyDrive/github_projects/data_dir/resumes/data/cv.md                                │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: CV Reader                                                                                               │
│                                                                                                                 │
│  Thought: Action: Read a file's content                                                                         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"/content/drive/MyDrive/github_projects/data_dir/resumes/data/cv.md\"}"                      │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  # John Doe                                                                                                     │
│                                                                                                                 │
│  ## Professional Summary                                                                                        │
│  Experienced Software Engineer with a strong background in Python, JavaScript, and RESTful APIs. Proven         │
│  ability to develop and maintain software applications, collaborate with cross-functional teams, and ensure     │
│  code quality.                                                                                                  │
│                                                                                                                 │
│  ## Technical Skills                                                                                            │
│  - **Programming Languages**: Python, JavaScript                                                                │
│  - **Web Development**: RESTful APIs, HTML, CSS                                                                 │
│  - **Frameworks/Libraries**: Django, Flask, React                                                               │
│  - **Tools**: Git, Docker, Jenkins                                                                              │
│  - **Databases**: MySQL, PostgreSQL                                                                             │
│                                                                                                                 │
│  ## Work History                                                                                                │
│                                                                                                                 │
│  ### Senior Software Engineer                                                                                   │
│  **Tech Innovations Inc., San Francisco, CA**                                                                   │
│  *January 2020 - Present*                                                                                       │
│  - Developed and maintained multiple web applications using Python and JavaScript.                              │
│  - Collaborated with cross-functional teams to design and implement RESTful APIs.                               │
│  - Ensured code quality through rigorous testing and code reviews.                                              │
│  - Automated deployment processes using Docker and Jenkins.                                                     │
│                                                                                                                 │
│  ### Software Engineer                                                                                          │
│  **Innovative Solutions LLC, San Francisco, CA**                                                                │
│  *June 2017 - December 2019*                                                                                    │
│  - Worked on full-stack web development projects using Python and JavaScript.                                   │
│  - Built and maintained RESTful APIs for various applications.                                                  │
│  - Participated in Agile development processes, including sprint planning and daily stand-ups.                  │
│  - Mentored junior developers and conducted code reviews.                                                       │
│                                                                                                                 │
│  ## Education                                         

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: CV Reader                                                                                               │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # John Doe                                                                                                     │
│                                                                                                                 │
│  ## Professional Summary                                                                                        │
│  Experienced Software Engineer with a strong background in Python, JavaScript, and RESTful APIs. Proven         │
│  ability to develop and maintain software applications, collaborate with cross-functional teams, and ensure     │
│  code quality.                                                                                                  │
│                                                                                                                 │
│  ## Technical Skills                                                                                            │
│  - **Programming Languages**: Python, JavaScript                                                                │
│  - **Web Development**: RESTful APIs, HTML, CSS                                                                 │
│  - **Frameworks/Libraries**: Django, Flask, React                                                               │
│  - **Tools**: Git, Docker, Jenkins                                                                              │
│  - **Databases**: MySQL, PostgreSQL                                                                             │
│                                                                                                                 │
│  ## Work History                                                                                                │
│                                                                                                                 │
│  ### Senior Software Engineer                                                                                   │
│  **Tech Innovations Inc., San Francisco, CA**                                                                   │
│  *January 2020 - Present*                                                                                       │
│  - Developed and maintained multiple web applications using Python and JavaScript.                              │
│  - Collaborated with cross-functional teams to design and implement RESTful APIs.                               │
│  - Ensured code quality through rigorous testing and code reviews.                                              │
│  - Automated deployment processes using Docker and Jenkins.                                                     │
│                                                                                                                 │
│  ### Software Engineer                                                                                          │
│  **Innovative Solutions LLC, San Francisco, CA**                                                                │
│  *June 2017 - December 2019*                                                                                    │
│  - Worked on full-stack web development projects using Python and JavaScript.                                   │
│  - Built and maintained RESTful APIs for various applications.                                                  │
│  - Participated in Agile development processes, includi

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Matcher                                                                                                 │
│                                                                                                                 │
│  Task: Match the CV to the job opportunities based on skills, experience, and key                               │
│      achievements.                                                                                              │
│      Evaluate how well the candidate's profile fits each job description,                                       │
│      focusing on the alignment of skills, work history, and key achievements                                    │
│      with the job requirements.                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
│      Jobs CSV file: /content/drive/MyDrive/github_projects/data_dir/resumes/data/jobs.csv                       │
│                                                                                                                 │
│      CV file: /content/drive/MyDrive/github_projects/data_dir/resumes/data/cv.md                                │
│      write the output in markdown format                                                                        │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Matcher                                                                                                 │
│                                                                                                                 │
│  Thought: Action: Read a file's content                                                                         │
│                                                                                                                 │
│  Using Tool: Read a file's content                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"file_path\": \"/content/drive/MyDrive/github_projects/data_dir/resumes/data/jobs.csv\"}"                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Job Title,Required Skills,Key Responsibilities,Company Name,Company Location                                   │
│  Software Engineer,"Python,JavaScript,RESTful APIs","Develop and maintain software applications, collaborate    │
│  with cross-functional teams, ensure code quality",TechCorp,San Francisco                                       │
│  Data Scientist,"Python,R,Machine Learning,Data Analysis","Analyze large datasets to extract insights, build    │
│  predictive models, collaborate with stakeholders",DataTech,New York                                            │
│  Project Manager,"Agile, Scrum,Leadership,Communication","Manage project timelines, coordinate with team        │
│  members, ensure project goals are met",ProjectPlus,Chicago                                                     │
│  DevOps Engineer,"Docker,Kubernetes,CI/CD","Implement and manage CI/CD pipelines, automate deployment           │
│  processes, monitor system performance",DevOpsCo,Seattle                                                        │
│  Marketing Specialist,"SEO,Content Creation,Social Media","Develop and execute marketing campaigns, create      │
│  content for various platforms, analyze campaign performance",MarketGenius,Los Angeles                          │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Matcher                                                                                                 │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Job Matching Results                                                                                         │
│                                                                                                                 │
│  ## Ranked Job Opportunities                                                                                    │
│                                                                                                                 │
│  ### 1. Job Title: Software Engineer                                                                            │
│  - **Match Score**: 9.5/10                                                                                      │
│  - **Key Matching Points**:                                                                                     │
│      - **Skills Alignment**: Strong match with Python, JavaScript, and RESTful APIs, all explicitly listed in   │
│  the candidate's technical skills.                                                                              │
│      - **Experience Alignment**: Extensive work history directly aligns with developing and maintaining         │
│  software applications, collaborating with cross-functional teams, and ensuring code quality through testing    │
│  and code reviews.                                                                                              │
│      - **Achievement Relevance**: The candidate's success in leading a system migration resulting in a 30%      │
│  performance increase, and implementing a CI/CD pipeline that reduced deployment times by 50%, directly         │
│  demonstrates high impact software engineering capabilities relevant to the role.                               │
│                                                                                                                 │
│  ### 2. Job Title: Project Manager                                                                              │
│  - **Match Score**: 8.0/10                                                                                      │
│  - **Key Matching Points**:                                                                                     │
│      - **Skills Alignment**: Direct experience with "Agile development processes" and demonstrated              │
│  "Leadership" through leading a project and mentoring junior developers. "Communication" is evident through     │
│  collaboration.                                                                                                 │
│      - **Experience Alignment**: The candidate's experience in leading a project and collaborating with teams   │
│  aligns well with managing project timelines, coordinating team members, and ensuring project goals are met.    │
│      - **Achievement Relevance**: Successfully leading a project to migrate a legacy system and achieving a     │
│  significant performance increase highlights strong project management and delivery capabilities.               │
│                                                                                                                 │
│  ### 3. Job Title: DevOps Engineer                                                                              │
│  - **Match Score**: 7.5/10                                                                                      │
│  - **Key Matching Points**:                            

In [62]:
print(result)

# Job Matching Results

## Ranked Job Opportunities

### 1. Job Title: Software Engineer
- **Match Score**: 9.5/10
- **Key Matching Points**:
    - **Skills Alignment**: Strong match with Python, JavaScript, and RESTful APIs, all explicitly listed in the candidate's technical skills.
    - **Experience Alignment**: Extensive work history directly aligns with developing and maintaining software applications, collaborating with cross-functional teams, and ensuring code quality through testing and code reviews.
    - **Achievement Relevance**: The candidate's success in leading a system migration resulting in a 30% performance increase, and implementing a CI/CD pipeline that reduced deployment times by 50%, directly demonstrates high impact software engineering capabilities relevant to the role.

### 2. Job Title: Project Manager
- **Match Score**: 8.0/10
- **Key Matching Points**:
    - **Skills Alignment**: Direct experience with "Agile development processes" and demonstrated "Leadershi